In [61]:
import os, csv, json, math, random, pathlib
import numpy as np
import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Performance: enable mixed precision when available (speeds up training on modern GPUs)
# try:
#     from tensorflow.keras import mixed_precision
#     mixed_precision.set_global_policy('mixed_float16')
#     print("Mixed precision enabled: float16 compute, float32 variables")
# except Exception as _e:
#     # fallback: mixed precision not available on this platform / TF build
#     print("Mixed precision not enabled (platform TF build may not support it):", _e)

# ==== CONFIG ====
H, W = 180, 180
IMAGE_SIZE = (H, W)
# Reduced BATCH for quick verification (was 64)
BATCH = 16
NUM_CLASSES = 2
CLASS_NAMES = ("Cat", "Dog")   # nomes como estão nas pastas
CLASS_MAP = {name: i for i, name in enumerate(CLASS_NAMES)}
INV_CLASS_MAP = {v:k for k,v in CLASS_MAP.items()}
# Limit unlabeled steps per epoch during quick checks (was 500)
MAX_U_STEPS = 50 # small value for fast verification; None = use all
DEBUG = True                  # True = logs verbosos, sem @tf.function no passo U
SHUFFLE_BUF_L = 1024          # buffers menores evitam longas “filling up…”
SHUFFLE_BUF_U = 1024
PREFETCH = tf.data.AUTOTUNE
LOG_EVERY_L = 50              # log a cada N batches L
LOG_EVERY_U = 50  

tau = 0.7          # threshold inicial mais brando
lambda_u_max = 1.0 # peso máximo da perda unsupervisionada
c_u, c_i = 1.0, 1.0  # coef. UCB
alpha = 0.9          # EMA

# Threading to avoid CPU oversubscription
# try:
#     tf.config.threading.set_inter_op_parallelism_threads(0)
#     tf.config.threading.set_intra_op_parallelism_threads(0)
# except Exception:
#     pass

# # Auto-adjust batch if GPU present (simple heuristic)
# _gpus = tf.config.list_physical_devices('GPU')
# if _gpus:
#     # try to increase batch multiplicatively to improve throughput
#     old_batch = BATCH
#     BATCH = int(BATCH * max(1, len(_gpus) * 2))
#     print(f"Detected {_gpus and len(_gpus)} GPU(s); increasing BATCH {old_batch} -> {BATCH}")
# else:
#     print("No GPUs detected; keeping default BATCH")

# # Cache decision will be computed after dataset listing (we may use psutil if available)
# CACHE_DATA = None  # None = decide automatically later, True/False forced


### Helpers de leitura e record builders

In [62]:
def _safe_decode_resize_jpeg(path, image_size):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3, dct_method='INTEGER_ACCURATE')
    img = tf.image.resize(img, image_size, antialias=True)
    img = tf.cast(img, tf.float32) / 255.0  # [0,1] — ÚNICA normalização
    return img

IGNORE_ERRORS = tf.data.experimental.ignore_errors()

def build_records_from_dir(root_dir, class_map=CLASS_MAP, with_ids=True, seed=42):
    """
    Varre .../Cat/*.jpg e .../Dog/*.jpg gerando records {sid, path, label}
    """
    rng = np.random.RandomState(seed)
    records = []
    sid = 0
    for name, idx in class_map.items():
        cls_dir = os.path.join(root_dir, name)
        for fn in os.listdir(cls_dir):
            if not fn.lower().endswith((".jpg", ".jpeg", ".png")):
                continue
            p = os.path.join(cls_dir, fn)
            rec = {"sid": sid, "path": p}
            if class_map is not None:
                rec["label"] = idx
            records.append(rec)
            sid += 1
    rng.shuffle(records)
    return records

def split_L_U(records, n_L=400, seed=42):
    rng = np.random.RandomState(seed)
    arr = records.copy()
    rng.shuffle(arr)
    L = []
    U = []
    for r in arr:
        if len(L) < n_L:
            L.append(r)
        else:
            U.append(r)
    return L, U


In [63]:
def _safe_decode_resize_jpeg(path, image_size):
    # path é tf.Tensor (scalar string)
    img_bytes = tf.io.read_file(path)

    # decode robusto a JPEG com headers estranhos
    img = tf.image.decode_jpeg(img_bytes, channels=3, dct_method='INTEGER_ACCURATE')

    # resize com antialias
    img = tf.image.resize(img, image_size, antialias=True)

    # cast e normalização AQUI (opção B)
    img = tf.cast(img, tf.float32) / 255.0  # <- garante [0,1]

    # (opcional) clip de segurança por ruído/artefatos
    img = tf.clip_by_value(img, 0.0, 1.0)
    return img


### Augmentations batch-first

In [64]:
weak_aug_seq = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomTranslation(0.05, 0.05),
], name="weak_aug")

strong_aug_seq = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.15),
    layers.RandomContrast(0.15),
], name="strong_aug")


In [65]:
# ==== AUGMENTS ================================================================
import tensorflow as tf
from tensorflow import keras

IMAGE_SIZE = (180, 180)  # ajuste ao seu valor

def make_weak_aug():
    # Flip + pequeno crop → mantém semântica (FixMatch "weak")
    return keras.Sequential([
        # keras.layers.RandomFlip("horizontal"),
        # keras.layers.RandomTranslation(height_factor=0.02, width_factor=0.02, fill_mode="reflect"),
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.05),
        keras.layers.RandomTranslation(height_factor=0.02, width_factor=0.02, fill_mode="reflect"),
        layers.Lambda(lambda x: tf.clip_by_value(x, 0.0, 1.0)),
    ])

def make_strong_aug(mild=False):
    if mild:
        # primeiras épocas: menos agressivo
        return keras.Sequential([
            keras.layers.RandomFlip("horizontal"),
            keras.layers.RandomRotation(0.15),
            keras.layers.RandomContrast(0.2),
            keras.layers.Lambda(lambda x: tf.clip_by_value(x, 0.0, 1.0)),
        ])
    # depois: mais agressivo
    return keras.Sequential([
        keras.layers.RandomFlip("horizontal"),
        keras.layers.RandomRotation(0.25),
        keras.layers.RandomContrast(0.4),
        keras.layers.RandomBrightness(factor=0.2),
        keras.layers.Lambda(lambda x: tf.clip_by_value(x, 0.0, 1.0)),
    ])

weak_aug   = make_weak_aug()
strong_aug = make_strong_aug(mild=True)   # começamos "mild"; trocaremos no loop


### Datasets (versões únicas, sem duplicidade de batch)

In [66]:
IGNORE_ERRORS = tf.data.experimental.ignore_errors()

def make_L_ds(L_records, batch, image_size, drop_remainder=False, cache=False):
    paths  = [r["path"] for r in L_records]
    labels = [r["label"] for r in L_records]
    ids    = [r["sid"]   for r in L_records]
    ds = tf.data.Dataset.from_tensor_slices((paths, labels, ids))

    def _load(path, y, sid):
        img = _safe_decode_resize_jpeg(path, image_size)  # [0,1]
        return img, y, sid

    ds = ds.shuffle(SHUFFLE_BUF_L, reshuffle_each_iteration=True)
    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    ds = ds.apply(IGNORE_ERRORS)

    # if cache:
    #     try:
    #         ds = ds.cache()
    #         print(f"L_ds: caching enabled (may use memory/disk)")
    #     except Exception as _e:
    #         print("L_ds: cache() failed:", _e)

    ds = ds.batch(batch, drop_remainder=drop_remainder)
    ds = ds.prefetch(PREFETCH)
    return ds

def make_U_ds(U_records, batch, image_size, drop_remainder=False,
              weak_aug=None, strong_aug=None, cache=False):
    paths = [r["path"] for r in U_records]
    ids   = [r["sid"]  for r in U_records]
    ds = tf.data.Dataset.from_tensor_slices((paths, ids))

    def _loadU(path, sid):
        img = _safe_decode_resize_jpeg(path, image_size)  # [0,1]
        return img, sid

    ds = ds.shuffle(SHUFFLE_BUF_U, reshuffle_each_iteration=True)
    ds = ds.map(_loadU, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    ds = ds.apply(IGNORE_ERRORS)

    # if cache:
    #     try:
    #         ds = ds.cache()
    #         print(f"U_ds: caching enabled (may use memory/disk)")
    #     except Exception as _e:
    #         print("U_ds: cache() failed:", _e)

    ds = ds.batch(batch, drop_remainder=drop_remainder)

    # aplica augment APÓS o batch
    def _make_pair(batch_img, batch_sid):
        # prefer aug functions/seq passados como argumento; fallback para globals
        weak = weak_aug if weak_aug is not None else weak_aug_seq
        strong = strong_aug if strong_aug is not None else strong_aug_seq

        x_w = weak(batch_img, training=True)
        x_s = strong(batch_img, training=True)
        # redundante se já há Lambda nas seqs, mas seguro:
        x_w = tf.clip_by_value(x_w, 0.0, 1.0)
        x_s = tf.clip_by_value(x_s, 0.0, 1.0)
        return x_w, x_s, batch_sid

    ds = ds.map(_make_pair, num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    ds = ds.prefetch(PREFETCH)
    return ds

def make_val_ds(val_records, batch, image_size):
    paths  = [r["path"] for r in val_records]
    labels = [r["label"] for r in val_records]
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))

    def _load(path, y):
        img = _safe_decode_resize_jpeg(path, image_size)  # [0,1]
        return img, y

    ds = ds.map(_load, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch).prefetch(tf.data.AUTOTUNE)
    return ds


### Modelo único com normalização no modelo (evita duplicidade)

In [67]:
def build_model(num_classes=NUM_CLASSES, image_size=IMAGE_SIZE):
    inp = keras.Input(shape=image_size + (3,))
    x = layers.Conv2D(32, 3, padding="same", activation="relu")(inp)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3, padding="same", activation="relu")(x)
    x = layers.Flatten()(x)
    out = layers.Dense(num_classes)(x)  # logits
    # ensure output is float32 even under mixed precision
    # out = layers.Activation('linear', dtype='float32')(out)
    return keras.Model(inp, out)

model = build_model()
# use optimizer with loss scaling if mixed precision is enabled
# try:
#     from tensorflow.keras import mixed_precision
#     if mixed_precision.global_policy().name == 'mixed_float16':
#         optimizer = mixed_precision.LossScaleOptimizer(keras.optimizers.Adam(1e-3))
#     else:
#         optimizer = keras.optimizers.Adam(1e-3)
# except Exception:
#     optimizer = keras.optimizers.Adam(1e-3)
optimizer = keras.optimizers.Adam(1e-3)

loss_obj  = keras.losses.SparseCategoricalCrossentropy(from_logits=True)


### Métricas por amostra, EMA/UCB

In [68]:
def softmax_logits(logits):
    return tf.nn.softmax(logits, axis=-1)

def pseudo_el2n(probs):  # incerteza (EL2N-like, proxy simples)
    # distância do one-hot do argmax
    yhat = tf.argmax(probs, axis=-1)
    oh = tf.one_hot(yhat, depth=tf.shape(probs)[-1])
    return tf.reduce_mean(tf.square(probs - oh), axis=-1)

def sym_kl(p, q, eps=1e-7):
    p = tf.clip_by_value(p, eps, 1.0)
    q = tf.clip_by_value(q, eps, 1.0)
    kl1 = tf.reduce_sum(p * tf.math.log(p/q), axis=-1)
    kl2 = tf.reduce_sum(q * tf.math.log(q/p), axis=-1)
    return 0.5*(kl1+kl2)

# tabelas EMA (legacy dicts kept for compatibility)
u_mean, u_var = {}, {}
i_mean, i_var = {}, {}

# tabelas EMA (GLOBAIS): arrays will be initialized after dataset listing for vectorized updates
U_MEAN, U_VAR = {}, {}
I_MEAN, I_VAR = {}, {}
U_MEAN_ARR = None
U_VAR_ARR = None
I_MEAN_ARR = None
I_VAR_ARR = None

def initialize_ema_arrays(n):
    """Create numpy arrays (float64) of length n for EMA/UCB statistics."""
    global U_MEAN_ARR, U_VAR_ARR, I_MEAN_ARR, I_VAR_ARR
    U_MEAN_ARR = np.zeros(n, dtype=np.float64)
    U_VAR_ARR  = np.zeros(n, dtype=np.float64)
    I_MEAN_ARR = np.zeros(n, dtype=np.float64)
    I_VAR_ARR  = np.zeros(n, dtype=np.float64)
    print(f"Initialized EMA arrays length={n}")

def ema_update_batch(mean_arr, var_arr, sids, vals, alpha=0.9):
    """Vectorized EMA update for arrays. sids and vals are numpy arrays or tensors."""
    if mean_arr is None or var_arr is None:
        return
    s = np.asarray(sids, dtype=np.int64)
    v = np.asarray(vals, dtype=np.float64)
    m = mean_arr[s]
    vv = var_arr[s]
    new_m = alpha*m + (1.0-alpha)*v
    new_v = alpha*vv + (1.0-alpha)*(v - new_m)**2
    mean_arr[s] = new_m
    var_arr[s] = new_v

def ema_update(mean_dict, var_dict, sid, val, alpha=0.9):
    # fallback scalar update (kept for compatibility)
    sid = int(sid)
    val = float(val)
    m = mean_dict.get(sid, val)
    v = var_dict.get(sid, 0.0)
    new_m = alpha*m + (1.0-alpha)*val
    new_v = alpha*v + (1.0-alpha)*(val - new_m)**2
    mean_dict[sid] = float(new_m)
    var_dict[sid]  = float(new_v)

def ucb(mu, var, c=1.0):
    return mu + c * math.sqrt(max(var, 1e-12))


In [69]:
def probe_confidences_on_U(model, U_ds, temp_T=0.5, 
                           max_batches=8,  # smaller probe for quick verification (was 32)
                           stop_if_n=2048   # stop earlier when enough confs collected
                           ):
    """
    Faz forward em alguns batches de U para medir confianças da WEAK.
    Retorna uma lista de confianças (max softmax).
    """
    confs = []
    n_batches = 0
    for x_w, _, _ in U_ds:  # U_ds -> (x_w, x_s, sid)
        logits_w = model(x_w, training=False)
        probs_w  = tf.nn.softmax(logits_w / temp_T, axis=-1)
        conf     = tf.reduce_max(probs_w, axis=-1).numpy()  # (B,)
        confs.append(conf)
        n_batches += 1
        if n_batches >= max_batches or sum(len(c) for c in confs) >= stop_if_n:
            break
    if len(confs) == 0:
        return []
    return np.concatenate(confs, axis=0)


### Passos de treino (supervisionado e não-rotulado) — sem dupla normalização

In [70]:
def train_step_supervised(batch):
    x_l, y_l, _ = batch
    start = time.time() if DEBUG else None
    with tf.GradientTape() as tape:
        logits_l = model(x_l, training=True)  # Rescaling acontece no modelo
        loss_sup = loss_obj(y_l, logits_l)

    grads = tape.gradient(loss_sup, model.trainable_variables)

    # If optimizer supports unscaling (LossScaleOptimizer), use it
    if hasattr(optimizer, 'get_unscaled_gradients'):
        try:
            grads = optimizer.get_unscaled_gradients(grads)
        except Exception:
            # some TF versions expose a different API; keep grads as-is
            pass

    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if DEBUG and start is not None:
        print(f"train_step_supervised: elapsed {time.time()-start:.4f} s")

    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(logits_l, -1), tf.cast(y_l, tf.int64)), tf.float32)
    )
    return loss_sup, acc


def train_step_unlabeled_with_masklog(batch, lambda_u=1.0, tau=0.7, temp_T=0.5):
    x_w, x_s, sids = batch
    start_total = time.time() if DEBUG else None

    # ---------- forward (fora do tape) para máscara e métricas ----------
    t0 = time.time() if DEBUG else None
    logits_w = model(x_w, training=True)
    probs_w  = tf.nn.softmax(logits_w / temp_T, axis=-1)
    if DEBUG and t0 is not None:
        print(f"unlabeled: forward weak took {time.time()-t0:.4f} s")

    # métricas (incerteza U, inconsistência I) e máscara
    conf = tf.reduce_max(probs_w, axis=-1)           # [B]
    yhat = tf.argmax(probs_w, axis=-1)               # [B], int
    mask = conf >= tau                               # [B], bool
    masked = tf.where(mask)                          # [M,1]
    masked_count = tf.shape(masked)[0]

    # Consistência com strong (para logging apenas – fora do tape)
    t1 = time.time() if DEBUG else None
    logits_s_for_I = model(x_s, training=True)
    probs_s_for_I  = tf.nn.softmax(logits_s_for_I, axis=-1)
    if DEBUG and t1 is not None:
        print(f"unlabeled: forward strong(for I) took {time.time()-t1:.4f} s")

    u_batch = pseudo_el2n(probs_w)                   # [B]
    i_batch = sym_kl(probs_w, probs_s_for_I)         # [B]

    # Sem exemplos válidos ou sem peso → não atualiza
    if (lambda_u <= 0.0) or tf.equal(masked_count, 0):
        if DEBUG and start_total is not None:
            print(f"unlabeled: skipped (no masked) elapsed {time.time()-start_total:.4f} s")
        return {
            "loss": tf.constant(0.0, tf.float32),
            "u_batch": u_batch,
            "i_batch": i_batch,
            "sids": sids,
            "mask_pass": tf.reduce_sum(tf.cast(mask, tf.int32)),
            "mask_total": tf.size(mask),
            "skipped": True,
        }

    # ---------- forward DENTRO do tape (grava computação para gradiente) ----------
    t2 = time.time() if DEBUG else None
    with tf.GradientTape() as tape:
        logits_s = model(x_s, training=True)                   # recomputa dentro do tape
        logits_s_mask = tf.gather(logits_s, masked[:, 0])      # [M, C]
        yhat_mask     = tf.gather(yhat,     masked[:, 0])      # [M]
        # perda de consistência (pseudo-rótulo)
        loss_unsup = loss_obj(yhat_mask, logits_s_mask)        # scalar
        loss = lambda_u * loss_unsup

        # handle mixed-precision loss scaling if optimizer provides it
        if hasattr(optimizer, 'get_scaled_loss'):
            try:
                scaled_loss = optimizer.get_scaled_loss(loss)
            except Exception:
                scaled_loss = loss
        else:
            scaled_loss = loss

    if DEBUG and t2 is not None:
        print(f"unlabeled: forward+loss(in tape) took {time.time()-t2:.4f} s")

    grads = tape.gradient(scaled_loss, model.trainable_variables)

    # If optimizer supports unscaling (LossScaleOptimizer), use it
    if hasattr(optimizer, 'get_unscaled_gradients'):
        try:
            grads = optimizer.get_unscaled_gradients(grads)
        except Exception:
            pass

    # Filtra gradientes None
    grads_vars = [(g, v) for g, v in zip(grads, model.trainable_variables) if g is not None]
    if not grads_vars:
        if DEBUG and start_total is not None:
            print(f"unlabeled: no grads (skipped) elapsed {time.time()-start_total:.4f} s")
        return {
            "loss": tf.constant(0.0, tf.float32),
            "u_batch": u_batch,
            "i_batch": i_batch,
            "sids": sids,
            "mask_pass": tf.reduce_sum(tf.cast(mask, tf.int32)),
            "mask_total": tf.size(mask),
            "skipped": True,
        }

    t3 = time.time() if DEBUG else None
    optimizer.apply_gradients(grads_vars)
    if DEBUG and t3 is not None and start_total is not None:
        print(f"unlabeled: apply_gradients took {time.time()-t3:.4f} s  total elapsed {time.time()-start_total:.4f} s")

    return {
        "loss": loss,
        "u_batch": u_batch,
        "i_batch": i_batch,
        "sids": sids,
        "mask_pass": tf.reduce_sum(tf.cast(mask, tf.int32)),
        "mask_total": tf.size(mask),
        "skipped": False,
    }

# Conditionally compile the train steps for speed when not in DEBUG
if not DEBUG:
    train_step_supervised = tf.function(train_step_supervised)
    train_step_unlabeled_with_masklog = tf.function(train_step_unlabeled_with_masklog)


In [71]:
# ==== SCHEDULES ==============================================================
def tau_schedule(epoch):
    # 0→0.6, 1→0.7, 2→0.8, saturando em 0.95 (se quiser, aumente depois)
    return min(0.95, 0.6 + 0.1*epoch)

def lambda_u_warmup(epoch, total=3, max_val=1.0):
    # sobe rápido para dar peso ao U cedo
    return max_val * min(1.0, (epoch+1)/total)

def choose_adaptive_tau(conf_list, target_rate=0.6, 
                        tau_min=0.40, tau_max=0.95,
                        prev_tau=None, smooth=0.6):
    """
    conf_list: lista/np.array de confianças (max softmax da weak)
    target_rate: fração de amostras que devem PASSAR na máscara (ex.: 0.6 -> 60%)
    tau_min/tau_max: limites de segurança
    prev_tau: tau da época anterior (para suavização exponencial)
    smooth: fator de suavização (0..1). 1.0 = sem suavização
    """
    if len(conf_list) == 0:
        # fallback conservador
        base = 0.5
        return float(prev_tau if prev_tau is not None else base)

    # Queremos que ~target_rate passem: tau = quantil de (1 - target_rate)
    q = float(np.clip(1.0 - target_rate, 0.0, 1.0))
    raw_tau = float(np.quantile(np.asarray(conf_list), q))
    raw_tau = float(np.clip(raw_tau, tau_min, tau_max))

    if prev_tau is None:
        return raw_tau
    # suavização exponencial: mais estável entre épocas
    return float(smooth * prev_tau + (1.0 - smooth) * raw_tau)


In [72]:
def run_one_round(L_ds, U_ds, epoch_idx, epochs=5, 
                  lambda_u_max=1.0, 
                  base_tau=0.60, 
                  temp_T=0.5,
                  target_rate_mild=0.60,   # ~60% passam na fase mild
                  target_rate_strong=0.50, # ~50% na fase forte
                  is_strong_phase=False,    # defina True/False fora conforme sua agenda mild→strong
                  prev_tau=None):           # opcional: passa o tau da época anterior p/ suavizar
    """
    Retorna o último tau usado (para você reutilizar/suavizar na próxima chamada).
    """
    tau_used = prev_tau if prev_tau is not None else base_tau

    for ep in range(epochs):
        # ---- 1) Supervisionado (igual)
        tL0 = time.time()
        l_loss_sum, l_acc_sum, l_batches = 0.0, 0.0, 0
        for batch in L_ds:
            loss_sup, acc = train_step_supervised(batch)
            l_loss_sum += float(loss_sup); l_acc_sum += float(acc); l_batches += 1
        tL1 = time.time()

        # ---- 2) Probe confianças da WEAK para escolher tau adaptativo desta época
        confs = probe_confidences_on_U(model, U_ds, temp_T=temp_T,
                                       max_batches=8, stop_if_n=2048)
        target_rate = (target_rate_strong if is_strong_phase else target_rate_mild)
        tau_used = choose_adaptive_tau(confs, target_rate=target_rate, 
                                       tau_min=0.40, tau_max=0.95,
                                       prev_tau=tau_used, smooth=0.6)

        # ---- 3) Não-rotulado (FixMatch) com tau adaptativo
        lam = lambda_u_warmup(ep, total=3, max_val=lambda_u_max)
        tU0 = time.time()
        u_batches, skipped_all = 0, 0
        pass_sum, total_sum = 0, 0
        u_mean_local, i_mean_local = 0.0, 0.0

        for batch in U_ds:
            out = train_step_unlabeled_with_masklog(
                batch, lambda_u=lam, tau=tau_used, temp_T=temp_T
            )
            u_batches += 1
            skipped_all += int(out["skipped"])

            # Respect debug short-run limit
            if MAX_U_STEPS is not None and u_batches >= MAX_U_STEPS:
                if DEBUG:
                    print(f"Reached MAX_U_STEPS={MAX_U_STEPS}; breaking U loop for debug")
                break

            # métricas
            pass_sum  += int(out["mask_pass"].numpy())
            total_sum += int(out["mask_total"].numpy())
            u_mean_local += float(tf.reduce_mean(out["u_batch"]))
            i_mean_local += float(tf.reduce_mean(out["i_batch"]))

            # EMA/UCB: vectorized when arrays exist
            u_vals  = out["u_batch"].numpy()
            i_vals  = out["i_batch"].numpy()
            sid_vals= out["sids"].numpy()
            if U_MEAN_ARR is not None:
                # vectorized batch update
                ema_update_batch(U_MEAN_ARR, U_VAR_ARR, sid_vals, u_vals, alpha=alpha)
                ema_update_batch(I_MEAN_ARR, I_VAR_ARR, sid_vals, i_vals, alpha=alpha)
            else:
                # fallback scalar updates
                for val_u, val_i, sid in zip(u_vals, i_vals, sid_vals):
                    ema_update(U_MEAN, U_VAR, int(sid), float(val_u), alpha=0.9)
                    ema_update(I_MEAN, I_VAR, int(sid), float(val_i), alpha=0.9)

        tU1 = time.time()

        mask_rate = (pass_sum / max(total_sum, 1)) * 100.0
        print(f"[epoch {epoch_idx}:{ep+1}/{epochs}]  "
              f"tau={tau_used:.2f}  λ_u={lam:.2f}  T={temp_T:.2f}  "
              f"mask_rate={mask_rate:.1f}%  skipped_batches={skipped_all}/{u_batches}  "
              f"L_loss~={l_loss_sum/max(l_batches,1):.4f}  L_acc~={l_acc_sum/max(l_batches,1):.3f}  "
              f"U~={u_mean_local/max(u_batches,1):.4f}  I~={i_mean_local/max(u_batches,1):.4f}  "
              f"({tL1-tL0:.1f}s L, {tU1-tU0:.1f}s U)")

    return tau_used


### Avaliação e execução de uma rodada

In [73]:
def evaluate(model, val_ds):
    tot, ok, loss_sum, n_batches = 0, 0, 0.0, 0
    for x, y in val_ds:
        logits = model(x, training=False)
        loss_sum += float(loss_obj(y, logits))
        ok += int((tf.argmax(logits, -1).numpy() == y.numpy()).sum())
        tot += int(y.shape[0])
        n_batches += 1
    mean_loss = loss_sum / max(n_batches, 1)
    acc = ok / max(tot, 1)
    return mean_loss, acc

# ==== LOOP DE UMA RODADA =====================================================
# def run_one_round(L_ds, U_ds, epoch_idx, epochs=5, lambda_u_max=1.0, temp_T=0.5):
#     import time
#     for ep in range(epochs):
#         current_tau = tau_schedule(ep)
#         lam = lambda_u_warmup(ep, total=3, max_val=lambda_u_max)

#         # (Opcional) logging rápido de confiança do 1º batch de U
#         try:
#             first_u = next(iter(U_ds))
#             xw, xs, _ = first_u
#             pw = tf.nn.softmax(model(xw, training=False) / temp_T, axis=-1)
#             conf = tf.reduce_max(pw, axis=-1).numpy()
#             conf_mean = float(conf.mean())
#             conf_p90  = float(np.percentile(conf, 90))
#         except StopIteration:
#             conf_mean, conf_p90 = float('nan'), float('nan')

#         print(f"[epoch {epoch_idx}:{ep+1}/{epochs}]  tau={current_tau:.2f}  λ_u={lam:.2f}  T={temp_T:.2f}  confμ={conf_mean:.3f}  confP90={conf_p90:.3f}")

#         # ---------- supervisionado ----------
#         tL = time.time()
#         L_batches, L_loss_sum, L_acc_sum = 0, 0.0, 0.0
#         for batch in L_ds:
#             loss_sup, acc = train_step_supervised(batch)
#             L_loss_sum += float(loss_sup)
#             L_acc_sum  += float(acc)
#             L_batches += 1
#         if L_batches:
#             print(f"  L | batches={L_batches}  loss~={L_loss_sum/L_batches:.4f}  acc~={L_acc_sum/L_batches:.3f}  ({time.time()-tL:.1f}s)")

#         # ---------- não-rotulado ----------
#         tU = time.time()
#         U_batches = 0
#         mask_pass_sum = 0
#         mask_total_sum = 0
#         skipped_batches = 0
#         U_u_sum = 0.0
#         U_i_sum = 0.0

#         for batch in U_ds:
#             out = train_step_unlabeled_with_masklog(batch, lambda_u=lam, tau=current_tau, T=temp_T)

#             U_batches += 1
#             mask_pass_sum  += int(out["mask_pass"].numpy())
#             mask_total_sum += int(out["mask_total"].numpy())
#             skipped_batches += int(bool(out["skipped"]))
#             U_u_sum += float(tf.reduce_mean(out["u_batch"]).numpy())
#             U_i_sum += float(tf.reduce_mean(out["i_batch"]).numpy())

#         if mask_total_sum > 0:
#             mask_rate = mask_pass_sum / mask_total_sum * 100.0
#         else:
#             mask_rate = 0.0

#         print(f"  U | batches={U_batches}  mask_rate={mask_rate:.1f}%  skipped={skipped_batches}  U~={U_u_sum/max(U_batches,1):.4f}  I~={U_i_sum/max(U_batches,1):.4f}  ({time.time()-tU:.1f}s)")

# def run_one_round(L_ds, U_ds, epoch_idx, epochs=5, 
#                   lambda_u_max=1.0, 
#                   base_tau=0.60, 
#                   temp_T=0.5,
#                   target_rate_mild=0.60,   # ~60% passam na fase mild
#                   target_rate_strong=0.50, # ~50% na fase forte
#                   is_strong_phase=False,    # defina True/False fora conforme sua agenda mild→strong
#                   prev_tau=None):           # opcional: passa o tau da época anterior p/ suavizar
#     """
#     Retorna o último tau usado (para você reutilizar/suavizar na próxima chamada).
#     """
#     tau_used = prev_tau if prev_tau is not None else base_tau

#     for ep in range(epochs):
#         # ---- 1) Supervisionado (igual)
#         tL0 = time.time()
#         l_loss_sum, l_acc_sum, l_batches = 0.0, 0.0, 0
#         for batch in L_ds:
#             loss_sup, acc = train_step_supervised(batch)
#             l_loss_sum += float(loss_sup); l_acc_sum += float(acc); l_batches += 1
#         tL1 = time.time()

#         # ---- 2) Probe confianças da WEAK para escolher tau adaptativo desta época
#         confs = probe_confidences_on_U(model, U_ds, temp_T=temp_T,
#                                        max_batches=32, stop_if_n=8192)
#         target_rate = (target_rate_strong if is_strong_phase else target_rate_mild)
#         tau_used = choose_adaptive_tau(confs, target_rate=target_rate, 
#                                        tau_min=0.40, tau_max=0.95,
#                                        prev_tau=tau_used, smooth=0.6)

#         # ---- 3) Não-rotulado (FixMatch) com tau adaptativo
#         lam = lambda_u_warmup(ep, total=3, max_val=lambda_u_max)
#         tU0 = time.time()
#         u_batches, skipped_all = 0, 0
#         pass_sum, total_sum = 0, 0
#         u_mean_local, i_mean_local = 0.0, 0.0

#         for batch in U_ds:
#             out = train_step_unlabeled_with_masklog(
#                 batch, lambda_u=lam, tau=tau_used, temp_T=temp_T
#             )
#             u_batches += 1
#             skipped_all += int(out["skipped"])

#             # métricas
#             pass_sum  += int(out["mask_pass"].numpy())
#             total_sum += int(out["mask_total"].numpy())
#             u_mean_local += float(tf.reduce_mean(out["u_batch"]))
#             i_mean_local += float(tf.reduce_mean(out["i_batch"]))

#             # EMA/UCB
#             u_vals  = out["u_batch"].numpy()
#             i_vals  = out["i_batch"].numpy()
#             sid_vals= out["sids"].numpy()
#             for val_u, val_i, sid in zip(u_vals, i_vals, sid_vals):
#                 ema_update(U_MEAN, U_VAR, int(sid), float(val_u), alpha=0.9)
#                 ema_update(I_MEAN, I_VAR, int(sid), float(val_i), alpha=0.9)

#         tU1 = time.time()

#         mask_rate = (pass_sum / max(total_sum, 1)) * 100.0
#         print(f"[epoch {epoch_idx}:{ep+1}/{epochs}]  "
#               f"tau={tau_used:.2f}  λ_u={lam:.2f}  T={temp_T:.2f}  "
#               f"mask_rate={mask_rate:.1f}%  skipped_batches={skipped_all}/{u_batches}  "
#               f"L_loss~={l_loss_sum/max(l_batches,1):.4f}  L_acc~={l_acc_sum/max(l_batches,1):.3f}  "
#               f"U~={u_mean_local/max(u_batches,1):.4f}  I~={i_mean_local/max(u_batches,1):.4f}  "
#               f"({tL1-tL0:.1f}s L, {tU1-tU0:.1f}s U)")

#     return tau_used




### Seleção, simulação de anotação e loop ASSL

In [74]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve() # Sobe um nível e resolve o caminho absoluto

# 2. Adiciona o caminho da raiz ao sys.path
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
    print(f"✅ Adicionado ao PYTHONPATH: {PROJECT_ROOT}")
else:
    print(f"📁 Pasta raiz já está no PYTHONPATH: {PROJECT_ROOT}")

📁 Pasta raiz já está no PYTHONPATH: C:\Users\felip\OneDrive\Documentos\GitHub\ActiveSemisupervisedLearningCNN-s


In [75]:
def acquire_topK_balanced(U_records, K, class_names):
    import time
    start = time.time()

    per_class = max(1, K // max(1, len(class_names)))
    buckets = {name: [] for name in class_names}

    for rec in U_records:
        sid = rec["sid"]
        p = rec["path"].replace("\\","/").lower()
        cname = None
        for name in class_names:
            if f"/{name.lower()}/" in p:
                cname = name; break
        if cname is None:
            continue
        # prefer array stats when available for speed
        # if U_MEAN_ARR is not None:
        #     mu_u, vu_u = float(U_MEAN_ARR[sid]), float(U_VAR_ARR[sid])
        # else:
        mu_u, vu_u = U_MEAN.get(sid, 0.0), U_VAR.get(sid, 0.0)
        # if I_MEAN_ARR is not None:
        #     mu_i, vi_i = float(I_MEAN_ARR[sid]), float(I_VAR_ARR[sid])
        # else:
        mu_i, vi_i = I_MEAN.get(sid, 0.0), I_VAR.get(sid, 0.0)
        score = ucb(mu_u, vu_u, c_u) * ucb(mu_i, vi_i, c_i)
        buckets[cname].append((score, sid))

    selected = []
    for cname in class_names:
        cand = sorted(buckets[cname], key=lambda x: x[0], reverse=True)
        selected.extend([sid for _, sid in cand[:per_class]])

    if len(selected) < K:
        rest = []
        have = set(selected)
        for rec in U_records:
            sid = rec["sid"]
            if sid in have: 
                continue
            if U_MEAN_ARR is not None:
                mu_u, vu_u = float(U_MEAN_ARR[sid]), float(U_VAR_ARR[sid])
            else:
                mu_u, vu_u = U_MEAN.get(sid, 0.0), U_VAR.get(sid, 0.0)
            if I_MEAN_ARR is not None:
                mu_i, vi_i = float(I_MEAN_ARR[sid]), float(I_VAR_ARR[sid])
            else:
                mu_i, vi_i = I_MEAN.get(sid, 0.0), I_VAR.get(sid, 0.0)
            rest.append((ucb(mu_u,vu_u,c_u)*ucb(mu_i,vi_i,c_i), sid))
        rest.sort(key=lambda x: x[0], reverse=True)
        selected.extend([sid for _, sid in rest[:K-len(selected)]])

    print(f"acquire_topK_balanced: elapsed {time.time()-start:.4f} s")
    return selected[:K]

def simulate_sid2true_from_path(U_records, L_records, class_map=CLASS_MAP):
    sid2true = {}
    def infer(path):
        p = path.replace("\\","/").lower()
        for name, idx in class_map.items():
            if f"/{name.lower()}/" in p:
                return idx
        raise ValueError(f"Não infere classe de: {path}")
    for rec in U_records: sid2true[rec["sid"]] = infer(rec["path"])
    for rec in L_records: sid2true[rec["sid"]] = infer(rec["path"])
    return sid2true


def move_annotated_to_L(sid2label, U_records, L_records):
    by_id = {r["sid"]: r for r in U_records}
    moved = []
    keepU = []
    for r in U_records:
        sid = r["sid"]
        if sid in sid2label:
            nr = dict(r)
            nr["label"] = int(sid2label[sid])
            moved.append(nr)
        else:
            keepU.append(r)
    return keepU, (L_records + moved)

# ==== Dataset base ====
train_root = r"..\data\train"
val_root   = r"..\data\validation"

all_train = build_records_from_dir(train_root, class_map=CLASS_MAP)
L_records, U_records = split_L_U(all_train, n_L=400, seed=42)

# initialize EMA arrays if possible (vectorized updates)
try:
    max_sid = max(r['sid'] for r in all_train)
    initialize_ema_arrays(max_sid+1)
except Exception as _e:
    print("Could not initialize EMA arrays:", _e)

# Decide caching heuristically (if psutil available)
# if CACHE_DATA is None:
#     try:
#         import psutil
#         avail = psutil.virtual_memory().available
#         # estimated RAM (float32) for all images in memory: 4 bytes * H * W * 3 channels * N
#         N = len(all_train)
#         est_bytes = 4 * H * W * 3 * N
#         # require <= 40% of available memory
#         CACHE_DATA = (est_bytes < (0.4 * avail))
#         print(f"Caching heuristic: est_bytes={est_bytes:,} avail={avail:,} -> CACHE_DATA={CACHE_DATA}")
#     except Exception as _e:
#         CACHE_DATA = False
#         print("psutil not available; defaulting CACHE_DATA=False")
CACHE_DATA=True

val_records = build_records_from_dir(val_root, class_map=CLASS_MAP)
val_ds = make_val_ds(val_records, batch=BATCH, image_size=IMAGE_SIZE)

CLASS_NAMES = list(CLASS_MAP.keys())  # ['Cat','Dog'] etc.
weak_aug   = make_weak_aug()
strong_aug = make_strong_aug(mild=True)
L_ds = make_L_ds(L_records, batch=BATCH, image_size=IMAGE_SIZE, cache=CACHE_DATA)
U_ds = make_U_ds(U_records, batch=BATCH, image_size=IMAGE_SIZE, cache=CACHE_DATA,
                 weak_aug=weak_aug, strong_aug=strong_aug)

# Checar L
xb, yb, _ = next(iter(L_ds))
print("L min/max:", float(tf.reduce_min(xb)), float(tf.reduce_max(xb)))

# Checar U (weak/strong)
xw, xs, _ = next(iter(U_ds))
print("U weak min/max:", float(tf.reduce_min(xw)), float(tf.reduce_max(xw)))
print("U strong min/max:", float(tf.reduce_min(xs)), float(tf.reduce_max(xs)))

# simulação de anotação (sem arquivos)
sid2true = simulate_sid2true_from_path(U_records, L_records, class_map=CLASS_MAP)

ROUNDS = 10
EPOCHS_PER_ROUND = 5

for r in range(ROUNDS):
    print(f"== Rodada {r+1}/{ROUNDS} ==")

    # --- primeiras épocas da rodada: mild ---
    strong_aug = make_strong_aug(mild=True)
    U_ds = make_U_ds(U_records, batch=BATCH, image_size=IMAGE_SIZE,
                     weak_aug=weak_aug, strong_aug=strong_aug, cache=CACHE_DATA)

    run_one_round(L_ds, U_ds, epoch_idx=r, epochs=2, lambda_u_max=1.0, temp_T=0.5)

    # --- depois, força total ---
    strong_aug = make_strong_aug(mild=False)
    U_ds = make_U_ds(U_records, batch=BATCH, image_size=IMAGE_SIZE,
                     weak_aug=weak_aug, strong_aug=strong_aug, cache=CACHE_DATA)

    run_one_round(L_ds, U_ds, epoch_idx=r, epochs=EPOCHS_PER_ROUND-2, lambda_u_max=1.0, temp_T=0.5)

    # avaliação
    vloss, vacc = evaluate(model, val_ds)
    print(f"Val loss: {vloss:.4f}  Val acc: {vacc:.4f}")

    # seleção ativa balanceada
    selected_ids = acquire_topK_balanced(U_records, K=40, class_names=CLASS_NAMES) # verificar qual a melhor qtd de amostra ideal

    # “anotação” simulada apenas nos K escolhidos
    sid2label = {sid: sid2true[sid] for sid in selected_ids}
    # mover U->L
    U_records, L_records = move_annotated_to_L(sid2label, U_records, L_records)

    # limpar EMA/UCB dos sids movidos
    moved_set = set(sid2label.keys())
    # if U_MEAN_ARR is not None:
    #     # zero out arrays for moved sids (cheap)
    #     for sid in moved_set:
    #         U_MEAN_ARR[sid] = 0.0; U_VAR_ARR[sid] = 0.0
    #         I_MEAN_ARR[sid] = 0.0; I_VAR_ARR[sid] = 0.0
    # else:
    for sid in list(U_MEAN.keys()):
        if sid in moved_set: U_MEAN.pop(sid, None); U_VAR.pop(sid, None)
    for sid in list(I_MEAN.keys()):
        if sid in moved_set: I_MEAN.pop(sid, None); I_VAR.pop(sid, None)

    # rebuild datasets
    L_ds = make_L_ds(L_records, batch=BATCH, image_size=IMAGE_SIZE, cache=CACHE_DATA)
    U_ds = make_U_ds(U_records, batch=BATCH, image_size=IMAGE_SIZE, cache=CACHE_DATA)


Initialized EMA arrays length=22502
L min/max: 0.0 1.0
L min/max: 0.0 1.0
U weak min/max: 0.0 1.0
U strong min/max: 0.0 1.0
== Rodada 1/10 ==
U weak min/max: 0.0 1.0
U strong min/max: 0.0 1.0
== Rodada 1/10 ==
train_step_supervised: elapsed 59.5304 s
train_step_supervised: elapsed 59.5304 s
train_step_supervised: elapsed 57.6342 s
train_step_supervised: elapsed 57.6342 s
train_step_supervised: elapsed 57.0532 s
train_step_supervised: elapsed 57.0532 s
train_step_supervised: elapsed 65.3285 s
train_step_supervised: elapsed 65.3285 s
train_step_supervised: elapsed 60.1511 s
train_step_supervised: elapsed 60.1511 s
train_step_supervised: elapsed 54.5186 s
train_step_supervised: elapsed 54.5186 s
train_step_supervised: elapsed 54.5945 s
train_step_supervised: elapsed 54.5945 s
train_step_supervised: elapsed 56.3752 s
train_step_supervised: elapsed 56.3752 s
train_step_supervised: elapsed 53.0958 s
train_step_supervised: elapsed 53.0958 s
train_step_supervised: elapsed 42.8052 s
train_step_

InvalidArgumentError: cannot compute Sub as input #1(zero-based) was expected to be a half tensor but is a float tensor [Op:Sub] name: 

In [ ]:
# TensorBoard profiler quick-run snippet (executa uma pequena carga e escreve perfil em logs/profile/)
import datetime
logdir = "logs/profile/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
print("Profiling to:", logdir)
try:
    tf.profiler.experimental.start(logdir)
    # execute uma rodada curta (1 rodada, poucas épocas) ou um pequeno trecho do pipeline
    run_one_round(L_ds, U_ds, epoch_idx=0, epochs=1, lambda_u_max=0.1, temp_T=0.5)
    tf.profiler.experimental.stop()
    print("Profiler finished. Use: tensorboard --logdir=logs/profile --port=6006")
except Exception as e:
    print("Profiler run failed:", e)
